# Text Files

# CSVs

As long as you work with data, you will encounter comma separated value (CSV) files.  CSV (Comma Separated Values) format is the most common import and export format for spreadsheets and databases. A csv file contains a number of rows, each containing a number of columns,
usually separated by commas.

Here, we'll explore how to use Python to read and parse csv files and then join that data to a spatial dataset. In our example, we will use data from [data.cms.gov](https://data.cms.gov/) on state by state enrollment into the Affordable Care Acts new [Pre-Existing Condition Insurance Plan (PCIP) program](https://data.cms.gov/Health/Monthly-Pre-Existing-Condition-Insurance-Plan-Enro/dpuq-z7nj) to make health insurance available to Americans denied coverage by private insurance companies because of a pre-existing condition. 

# csv module

The csv module's purpose is to make it easier to deal with csv formatted file, especially when working with data exported from spreadsheets and databases into text files. First, let's import the csv module.

In [ ]:
import csv

reading a csv file looks a lot like reading a text file. Let's read the monthly pre-existing conditionenrollment csv file and print out all of the rows.

In [ ]:
csvfile = r"C:\Users\greg6750\Documents\IPython Notebooks\Python_for_GIS_and_RS\Week_3\data\Monthly_Pre-Existing_Condition_Insurance_Plan_Enrollment__State_by_State.csv"

In [ ]:
with open(csvfile, 'rt') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

The 'rt' tells the csv reader that we are looking at a text file. 

Notice that the header of the csv file also gets printed out. Let's use a slightly different technique to read the file in. Also, Let's separate the header from the rest of the data and set it up so that per state, we print our the monthly enrollment.

In [ ]:
ifile  = open(csvfile, "rt")
reader = csv.reader(ifile)

rownum = 0
for row in reader:
    # Save header row.
    if rownum == 0:
        header = row
    else:
        colnum = 0
        for col in row:
            print(header[colnum] + ": " + str(col))
            colnum += 1
            
    rownum += 1

Here, we went through and for every state, we listed out the number of enrollees per month.

## Quetion: What is the most recent month for which there is data?

# CSV to ArcGIS Table

Arcpy has a very nice function called [Table to Table](http://pro.arcgis.com/en/pro-app/tool-reference/conversion/table-to-table.htm) that will convert a CSV directly to a table in a geodatabase. This is very helpful when we want to make a non-spatial spreadsheet a spatial dataset. Let's use arcpy and Table to Table to import our ACA data. Let's also import os so that we can create the output geodatabase that will contain the table.

In [ ]:
import arcpy
import os

In [ ]:
output_gdb = r'C:\Users\greg6750\Documents\IPython Notebooks\Python_for_GIS_and_RS\Week_3\data\aca.gdb'

#Use os to split the output_gdb into a name and path
gdb_name = os.path.basename(output_gdb)
gdb_path = os.path.dirname(output_gdb)

#print the name and path to verifiy that they are correct
print(gdb_name)
print(gdb_path)

Let's use arcpy to create a file geodatabase from the gdb_path and gdb_name.

In [ ]:
arcpy.CreateFileGDB_management(gdb_path, gdb_name)

Now that we've created a geodatabase, we can use [Table to Table](http://pro.arcgis.com/en/pro-app/tool-reference/conversion/table-to-table.htm) to import the CSV file into the geodatabase as a table. Let's go ahead and do that.

In [ ]:
table_name = 'enrollment_by_month'
arcpy.TableToTable_conversion(csvfile, output_gdb, table_name)

## Question: Did it work? Open ArcMap and check!

You can use arcpy to verify that it worked. Let's do a quick check.

In [ ]:
arcpy.env.workspace = output_gdb
arcpy.GetCount_management(table_name)

## Question: How many rows are in the table? Is this what you expected?

# Joining the Table to a Feature Class

Now that we have imported the CSV into a table, we can very easily join it to a feature class. Next, we'll do just that. We will join the table we created with a USA states feature class in order to illustrate how we can use python to automate the whole workflow of converting a CSV to a feature class.

In the Week_3\data\states_21basic folder there is a shapefile named states.shp. Let's look at that shapefile in ArcMap.
1. Open ArcMap
2. Add states.shp to the map.
3. Open the attribute table. In the attribute table, you should see a field called STATE_NAME. Verify that this is true.
4. Find you aca.gbd geodatabase. Add the enrollment_by_month table to the ArcMap Table of contents.
5. Open the enrollment_by_month table. You should see a State field. Verify that this is true.

Now we will use the [Join Field](http://pro.arcgis.com/en/pro-app/tool-reference/data-management/join-field.htm) to join the table to the states.shp shapefile. If you haven't imported arcpy yet, import arcpy. Then, let's join the data together.

In [ ]:
import arcpy

states = r'C:\Users\greg6750\Documents\IPython Notebooks\Python_for_GIS_and_RS\Week_3\data\states_21basic\states.shp'
state_join_field = 'STATE_NAME'
table = r'C:\Users\greg6750\Documents\IPython Notebooks\Python_for_GIS_and_RS\Week_3\data\aca.gdb\enrollment_by_month'
table_join_field =  'State'

arcpy.JoinField_management(states, state_join_field, table, table_join_field)

## Question: Did it Work? Open ArcMap and verify. You may notice the some of the field names changed. Do you know why?